# Install necessary dependencies

In [ ]:
#!git clone https://github.com/NVIDIA/apex && cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
!pip install --upgrade efficientnet-pytorch
!pip install --upgrade pretrainedmodels
!pip install git+https://github.com/qubvel/segmentation_models.pytorch
!pip install git+https://github.com/michal-nahlik/FastFCN

# Prepare
## Imports

In [ ]:
#from apex import amp

import os
import cv2
import math
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import albumentations as albu

from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.optimizer import Optimizer, required
from torch.utils.data import Dataset, DataLoader

import pretrainedmodels
from efficientnet_pytorch import EfficientNet
from segmentation_models_pytorch import Unet, FPN, PSPNet
from encoding.models.encnet import EncNet
from encoding.models.deeplabv3 import DeepLabV3

## Set seeds

In [ ]:
seed = 2019
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

## Dataset

In [ ]:
class CloudDataset(Dataset):
    def __init__(self, list_IDs=None, rles_df = None, data_folder = None, transforms=None,
                dim=(1400, 2100), reshape=(320, 480)):
        self.list_IDs = list_IDs
        self.rles_df = rles_df
        self.data_folder = data_folder
        self.transforms = transforms
        self.dim = dim
        self.reshape = reshape
        

    def __getitem__(self, idx):
        ID = self.list_IDs.iloc[idx]
        img_path = self.data_folder + ID
        img = self.load_rgb(img_path)
        mask = None
        
        if self.reshape is not None:
            img = np_resize(img, self.reshape)
            
        if self.rles_df is not None:
            image_df = self.rles_df[self.rles_df['ImageId'] == ID]
            rles = image_df['EncodedPixels'].values
            
            if self.reshape is not None:
                mask = build_masks(rles, input_shape=self.dim, reshape=self.reshape)
            else:
                mask = build_masks(rles, input_shape=self.dim)
        
        if self.transforms is not None:
            augmented = self.transforms(image=img, mask=mask)
            img  = augmented["image"]
            mask = augmented["mask"]
        
        return img, mask

    def __len__(self):
        return len(self.list_IDs)
    
    def load_rgb(self, img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.

        return img

## Helper functions

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def dice(img1, img2):
    img1 = img1 > 0.5
    img2 = img2 > 0.5
    img1 = np.asarray(img1).astype(np.bool)
    img2 = np.asarray(img2).astype(np.bool)

    intersection = np.logical_and(img1, img2)

    return 2.0 * intersection.sum() / (img1.sum() + img2.sum())

## rle, mask, image manipulation

In [ ]:
def np_resize(img, input_shape):
    """
    Reshape a numpy array, which is input_shape=(height, width), 
    as opposed to input_shape=(width, height) for cv2
    """
    height, width = input_shape
    return cv2.resize(img, (width, height))


def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def rle2mask(rle, input_shape):
    width, height = input_shape[:2]
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
        current_position += lengths[index]
        
    return mask.reshape(height, width).T


def build_masks(rles, input_shape, reshape=None):
    depth = len(rles)
    if reshape is None:
        masks = np.zeros((*input_shape, depth))
    else:
        masks = np.zeros((*reshape, depth))
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            if reshape is None:
                masks[:, :, i] = rle2mask(rle, input_shape)
            else:
                mask = rle2mask(rle, input_shape)
                reshaped_mask = np_resize(mask, reshape)
                masks[:, :, i] = reshaped_mask
    
    return masks


def build_rles(masks, thrs=None, reshape=None):
    width, height, depth = masks.shape
    
    rles = []
    
    for i in range(depth):
        mask = masks[:, :, i]
        if thrs is not None:
            mask = mask > thrs[i]
        
        if reshape:
            mask = mask.astype(np.float32)
            mask = np_resize(mask, reshape).astype(np.int64)
        
        rle = mask2rle(mask)
        rles.append(rle)
        
    return rles


def rle_area(rle):
    try:
        array = np.asarray([int(x) for x in rle.split()])
        lengths = array[1::2]
        return np.sum(lengths)
    except:
        return 0

## Transformations

In [ ]:
# Taken from newer version of albumentations, changed apply_to_mask transpose so the multilayer mask works
from albumentations.core.transforms_interface import BasicTransform

class ToTensorV2(BasicTransform):
    """Convert image and mask to `torch.Tensor`."""

    def __init__(self, always_apply=True, p=1.0):
        super(ToTensorV2, self).__init__(always_apply=always_apply, p=p)

    @property
    def targets(self):
        return {"image": self.apply, "mask": self.apply_to_mask}

    def apply(self, img, **params):
        return torch.from_numpy(img.transpose(2, 0, 1))

    def apply_to_mask(self, mask, **params):
        return torch.from_numpy(mask.transpose(2, 0, 1))

    def get_transform_init_args_names(self):
        return []

    def get_params_dependent_on_targets(self, params):
        return {}

In [ ]:
import cv2

from albumentations import (
    Compose, VerticalFlip, HorizontalFlip, ShiftScaleRotate, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,OneOf,
    ToFloat, ShiftScaleRotate,GridDistortion, ElasticTransform, JpegCompression, HueSaturationValue,
    RGBShift, RandomBrightness, RandomContrast, Blur, MotionBlur, MedianBlur, GaussNoise,CenterCrop,
    IAAAdditiveGaussianNoise,GaussNoise,OpticalDistortion,RandomSizedCrop
)

AUGMENTATIONS_TRAIN = Compose([
    OneOf([
        RandomContrast(),
        RandomGamma(),
        RandomBrightness(),
         ], p=0.3),
    OneOf([
        OpticalDistortion(distort_limit=0.5, shift_limit=0.5),
        ], p=0.3),
    OneOf([
        HorizontalFlip(),
        VerticalFlip(),
        ], p=0.3),
    ShiftScaleRotate(scale_limit=0.05, rotate_limit=25, p=0.3),
    ToFloat(max_value=1),
    ToTensorV2()
],p=1)


AUGMENTATIONS_TEST = Compose([
    ToFloat(max_value=1),
    ToTensorV2()
],p=1)

# Load and prepare data 

In [ ]:
class_names = ["Fish", "Flower", "Gravel", "Sugar"]

In [ ]:
path = '../input/understanding_cloud_organization/'
path_train = path + 'train_images/'
path_test = path + 'test_images/'

train_on_gpu = torch.cuda.is_available()

In [ ]:
train_df = pd.read_csv(path + 'train.csv')
train_df['ImageId'] = train_df['Image_Label'].apply(lambda x: x.split('_')[0])
train_df['ClassId'] = train_df['Image_Label'].apply(lambda x: x.split('_')[1])
train_df['MaskArea'] = train_df['EncodedPixels'].apply(lambda x: rle_area(x))
train_df['hasMask'] = ~ train_df['EncodedPixels'].isna()

print(train_df.shape)
train_df.head()

In [ ]:
mask_count_df = train_df.groupby('ImageId').agg({'hasMask' : np.sum, 'MaskArea': list}).reset_index()
print(np.shape(mask_count_df))
mask_count_df.head()

## Split based on mask area

In [ ]:
mask_areas = np.stack(mask_count_df['MaskArea'].values)
mask_areas = mask_areas > 0
train_idx, val_idx = train_test_split(mask_count_df['ImageId'], stratify=mask_areas, random_state=seed, test_size=0.2)

In [ ]:
train_mask_areas = np.stack(mask_count_df.iloc[train_idx.index]['MaskArea'].values)
val_mask_areas   = np.stack(mask_count_df.iloc[val_idx.index]['MaskArea'].values)

f, ax = plt.subplots(nrows=2, ncols=4, figsize=(20,8))
for i in range(0,4):
    sns.distplot(train_mask_areas[:,i], ax=ax[0,i]).set_title(class_names[i])
    sns.distplot(val_mask_areas[:,i], ax=ax[1,i])

In [ ]:
print('Training set size: {}'.format(len(train_idx)))
print('Validation set size: {}'.format(len(val_idx)))

## Create data loaders

In [ ]:
# define dataset and dataloader
num_workers = 4
bs = 8

train_dataset = CloudDataset(list_IDs = train_idx, rles_df=train_df, data_folder = path_train, transforms=AUGMENTATIONS_TRAIN)
valid_dataset = CloudDataset(list_IDs = val_idx,   rles_df=train_df, data_folder = path_train, transforms=AUGMENTATIONS_TEST)

train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=num_workers)
valid_loader = DataLoader(valid_dataset, batch_size=bs, shuffle=False, num_workers=num_workers)

In [ ]:
f, ax = plt.subplots(ncols=5,nrows=5, figsize=(24,20))

for i in range(5):
    image, mask = train_dataset.__getitem__(i)

    ax[i][0].imshow(image.numpy().transpose(1,2,0))
    for j in range(4):      
        ax[i][j + 1].imshow(mask[j, ...])
        ax[i][j + 1].set_title(class_names[j])

# Create and train model

## RAdam optimizer
source: https://github.com/LiyuanLucasLiu/RAdam/blob/master/radam.py

In [ ]:
class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:            
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)

                p.data.copy_(p_data_fp32)

        return loss

## Model

In [ ]:
device = torch.device("cuda:0")

In [ ]:
model_name = 'unet'
model = Unet("efficientnet-b4", encoder_weights='imagenet', classes=4)

#model_name = 'encnet'
#model = EncNet(4, 'resnet50', jpu=True, lateral=False, norm_layer_1d=nn.BatchNorm1d, norm_layer_2d=nn.BatchNorm2d)

#model_name = 'deeplabv3'
#model = DeepLabV3(4, 'resnet50', aux=True, se_loss=False, norm_layer=nn.BatchNorm2d)

#model_name = 'fpn'
#model = FPN("efficientnet-b4", encoder_weights='imagenet', classes=4)

if train_on_gpu:
    model.to(device)

## Metric and loss definition

In [ ]:
def dice_no_threshold(
    outputs: torch.Tensor,
    targets: torch.Tensor,
    eps: float = 1e-7,
    threshold: float = None,
):

    outputs = sigmoid(outputs)

    if threshold is not None:
        outputs = (outputs > threshold).float()

    intersection = torch.sum(targets * outputs)
    union = torch.sum(targets) + torch.sum(outputs)
    dice = 2 * intersection / (union + eps)

    return dice

def dice_loss(input, target):
    input = torch.sigmoid(input)
    smooth = 1.0
    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    return ((2.0 * intersection + smooth) / (iflat.sum() + tflat.sum() + smooth))


class FocalLoss(nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma

    def forward(self, input, target):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))
        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()
        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        return loss.mean()


class MixedLoss(nn.Module):
    def __init__(self, alpha, gamma):
        super().__init__()
        self.alpha = alpha
        self.focal = FocalLoss(gamma)

    def forward(self, input, target):
        loss = self.alpha*self.focal(input, target) - torch.log(dice_loss(input, target))
        return loss.mean()

In [ ]:
criterion = MixedLoss(10.0, 2.0)
optimizer = RAdam(model.parameters(), lr = 0.005)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.2, patience=2, cooldown=2)

In [ ]:
#amp.initialize(model, optimizer, opt_level='O1')

# use the following backward pass in training instead of loss.backward() in case of using apex 
#with amp.scale_loss(loss, optimizer) as scaled_loss:
#    scaled_loss.backward()


## Train model

In [ ]:
n_epochs = 35
train_loss_list = []
valid_loss_list = []
dice_score_list = []
lr_rate_list = []

valid_loss_min = np.Inf
valid_dice_max = 0

In [ ]:
for epoch in range(1, n_epochs+1):
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    dice_score = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    bar = tqdm(train_loader, postfix={"train_loss":0.0})
    for data, target in bar:
        # move data to GPU if CUDA is available
        if train_on_gpu:
            data = data.to(device, dtype=torch.float)
            target = target.to(device, dtype=torch.float)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item() * data.size(0)
        bar.set_postfix(ordered_dict={"train_loss":loss.item()})
    
    ######################    
    # validate the model #
    ######################
    model.eval()
    
    del data, target
    with torch.no_grad():
        bar = tqdm(valid_loader, postfix={"valid_loss":0.0, "dice_score":0.0})
        for data, target in bar:
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data = data.to(device, dtype=torch.float)
                target = target.to(device, dtype=torch.float)
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss += loss.item()*data.size(0)
            dice_cof = dice_no_threshold(output.cpu(), target.cpu()).item()
            dice_score +=  dice_cof * data.size(0)
            bar.set_postfix(ordered_dict={"valid_loss":loss.item(), "dice_score":dice_cof})
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    dice_score = dice_score/len(valid_loader.dataset)
    train_loss_list.append(train_loss)
    valid_loss_list.append(valid_loss)
    dice_score_list.append(dice_score)
    lr_rate_list.append([param_group['lr'] for param_group in optimizer.param_groups])
    
    # print training/validation statistics 
    print('Epoch: {}  Training Loss: {:.6f}  Validation Loss: {:.6f} Dice Score: {:.6f}'.format(
        epoch, train_loss, valid_loss, dice_score))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model, '{}_clouds_loss.pth'.format(model_name))
        valid_loss_min = valid_loss
        
    # save model if dice score increased
    if dice_score > valid_dice_max:
        print('Dice score increased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_dice_max,
        dice_score))
        torch.save(model, '{}_clouds_dice.pth'.format(model_name))
        valid_dice_max = dice_score
    
    scheduler.step(valid_loss)

In [ ]:
model = torch.load('./{}_clouds_dice.pth'.format(model_name))

### Show training history

In [ ]:
plt.figure(figsize=(10,10))
plt.plot([i[0] for i in lr_rate_list], marker='o')
plt.ylabel('Learing rate during training', fontsize=22)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(train_loss_list,  marker='o', label="Training Loss")
plt.plot(valid_loss_list,  marker='o', label="Validation Loss")
plt.ylabel('loss', fontsize=22)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(dice_score_list, marker='o')
plt.ylabel('Dice score')
plt.show()

# Show model output

In [ ]:
data, target = next(iter(valid_loader))
data = data.to(device, dtype=torch.float)
output = model(data)

In [ ]:
data = data.cpu().detach().numpy()
target = target.detach().numpy()
output = output.cpu().detach().numpy()

In [ ]:
for i in range(5):
    f, ax = plt.subplots(ncols=5, nrows=2, figsize=(20,4))
    ax[0][0].imshow(data[i, ...].transpose(1,2,0))
    ax[1][0].imshow(data[i, ...].transpose(1,2,0))

    ax[0][0].set_ylabel('Target')
    ax[1][0].set_ylabel('Output')
    
    for j in range(0, 4):
        ax[0][j + 1].set_title(class_names[j])
        ax[0][j + 1].imshow(target[i, j, :, :])
        ax[1][j + 1].imshow(output[i, j, :, :])

In [ ]:
# Apex folder has to be deleted otherwise Kaggle kernel crashes due too many files
#!rm -rf /kaggle/working/apex